In [1]:
"""
   Copyright 2023 Raphaël Isvelin

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""

'\n   Copyright 2023 Raphaël Isvelin\n\n   Licensed under the Apache License, Version 2.0 (the "License");\n   you may not use this file except in compliance with the License.\n   You may obtain a copy of the License at\n\n       http://www.apache.org/licenses/LICENSE-2.0\n\n   Unless required by applicable law or agreed to in writing, software\n   distributed under the License is distributed on an "AS IS" BASIS,\n   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n   See the License for the specific language governing permissions and\n   limitations under the License.\n'

## Jupyter CadQuery - Config and setup

In [2]:
VIEWER_THEME = "light"  # light, dark
VIEWER_HEIGHT = 900
ENABLE_REPLAY = True

In [3]:
import cadquery as cq

from jupyter_cadquery import show as jcq_show
from jupyter_cadquery import (
    versions,
    PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer, get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
enable_replay(ENABLE_REPLAY)

from cq_warehouse.fastener import (
    polygon_diagonal,
    read_fastener_parameters_from_csv,
    Screw, SocketHeadCapScrew, HeatSetNut, PlainWasher,
    HexNutWithFlange, HexHeadScrew, PanHeadScrew, CounterSunkScrew
)
import cq_warehouse.extensions

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay
Note: To get rid of this warning, use 'enable_replay(False)'


## show() improved

In [4]:
cv = open_viewer("Assembly", anchor="right")  # sets default viewer

In [5]:
objects_to_hide = [
    "/Masks"
]

def hide_objects_matching_strings(strings, keep_edges=True, cv=cv):
    for state in cv.widget.states:
        should_hide = False
        for s in strings:
            if s in state:
                should_hide = True
                break
        if should_hide:
            cv.update_states({
                state: (0, 0 if keep_edges else 0),
            })

In [6]:
def show(obj, viewer="Assembly", anchor="right", hide_contains=objects_to_hide):
    cv = jcq_show(
        obj,
        viewer=viewer,
        anchor=anchor,
        #cad_width=1640,
        height=VIEWER_HEIGHT,
        theme=VIEWER_THEME,
        collapse=1,
        optimal_bb=True,
        render_edges=True,
        axes=True,
        axes0=True,
        grid=[True, True, True],
        #black_edges=True,
        reset_camera=False,
        #show_parent=True,
        #timeit=True,
        #js_debug=True
    )
    hide_objects_matching_strings(hide_contains, cv=cv)

def show_b(obj, viewer="Quick", hide_contains=objects_to_hide):
    cv2 = open_viewer(viewer, anchor="split-bottom")
    show(obj, viewer, anchor="split-bottom", hide_contains=hide_contains)

def show_x(obj, hide_contains=objects_to_hide):
    show(obj, viewer=None, hide_contains=hide_contains)

### Show part (debug)

In [7]:
def show_part(part: Part, bottom=False):
    footprint_assembly = (
        cq.Assembly()
            .add(part.debug_objects.footprint.inside, name="Inside", color=cq.Color(1, 0, 1))
            .add(part.debug_objects.footprint.outside, name="Outside", color=cq.Color(0, 1, 1))
    )
    test_assembly = (
        cq.Assembly(None, name="test_assembly")
            .add(part.mask, name="Mask", color=cq.Color(0, 1, 0))
            .add(part.part, name="Part")
            .add(part.assembly_parts_to_cq_assembly(), name="Part assembly")
            .add(footprint_assembly, name="Footprint")
    )
    debug_assembly = cq.Assembly()
    debug_assembly.add(part.debug_objects.hole, name="Hole", color=cq.Color(1, 0, 0))
    for debug_obj_name in part.debug_objects.others.keys():
        obj = part.debug_objects.others[debug_obj_name]
        if not isinstance(obj, dict):
            debug_assembly.add(obj, name=debug_obj_name, color=cq.Color(0.5, 0.7, 0.5))
    test_assembly.add(debug_assembly, name="Debug")

    if bottom: show_b(test_assembly)
    else: show(test_assembly)

In [8]:
import sys
sys.path.append("../src")

from cq_enclosure_builder import PartFactory as pf
from cq_enclosure_builder import Panel, PanelSize
from cq_enclosure_builder import Enclosure, EnclosureSize, Face

## Sandbox / Testing

In [9]:
pf.set_default_types({
    "jack": '6.35mm PJ-612A',
    "usb_a": '3.0 vertical cltgxdd',
    "usb_c": 'ChengHaoRan E',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
    "screen": 'HDMI 5 inch JRP5015',
    "potentiometer": 'WH148',
    "banana": '4mm',
    "barrel_plug": 'DC-022B',
    "rca": 'N1030',
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2
})

usb_c = pf.build_usb_c()
usb_c_v = pf.build_usb_c(orientation_vertical=True)
usb_a = pf.build_usb_a()
usb_a_v = pf.build_usb_a(orientation_vertical=True)
spst = pf.build_button()
encoder = pf.build_encoder() 
jack_35 = pf.build_jack(part_type='3.5mm XXX')
jack_635 = pf.build_jack()
screen = pf.build_screen()
pot = pf.build_potentiometer()
banana = pf.build_banana()
barrel_plug = pf.build_barrel_plug()
rca = pf.build_rca()


VALIDATING CLASS: UsbCChengHaoRanEPart
VALIDATING CLASS: UsbCChengHaoRanEPart
VALIDATING CLASS: UsbA30VerticalCltgxddPart
VALIDATING CLASS: UsbA30VerticalCltgxddPart
VALIDATING CLASS: ButtonSpstPbs24b4Part
VALIDATING CLASS: EncoderEc11Part
VALIDATING CLASS: Jack3_5mmXxxPart
VALIDATING CLASS: Jack6_35mmPj612aPart
TODO Need to move all workplanes so the hole's center is at 0,0
VALIDATING CLASS: Hdmi5InchJrp5015Part
VALIDATING CLASS: PotentiometerWh148Part
VALIDATING CLASS: Banana4mmPart
VALIDATING CLASS: BarrelPlugDc022bPart
VALIDATING CLASS: RcaN1030Part


In [9]:
import sys
sys.path.append("../src")

In [9]:
ENCLOSURE_INNER_WIDTH =  220
ENCLOSURE_INNER_LENGTH = 150
ENCLOSURE_OUTER_THICKNESS = 38
ENCLOSURE_WALLS_THICKNESS = 2

from cq_enclosure_builder import PartFactory as pf

pf.set_default_types({
    "jack": '6.35mm PJ-612A',
    "usb_a": '3.0 vertical cltgxdd',
    "usb_c": 'ChengHaoRan E',
    "button": 'SPST PBS-24B-4',
    "encoder": 'EC11',
    "screen": 'HDMI 5 inch JRP5015',
    "air_vent": 'basic rectangular',
    "potentiometer": 'WH148',
    "banana": '4mm',
    "barrel_plug": 'DC-022B',
    "rca": 'N1030',
    "support": "pyramid"
})
pf.set_default_parameters({
    "enclosure_wall_thickness": 2
})

enclosure_size = EnclosureSize(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_OUTER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
enclosure = Enclosure(enclosure_size)
enclosure.add_part_to_face(Face.TOP, "Screen", pf.build_screen(), rel_pos=(0, -4))
# enclosure.add_part_to_face(Face.TOP, "SPST 1", pf.build_button(), abs_pos=(20, 30))
# enclosure.add_part_to_face(Face.TOP, "SPST 2", pf.build_button(), abs_pos=(20, 100))
enclosure.add_part_to_face(Face.TOP, "SPST 3", pf.build_button(), abs_pos=(15, 10))
enclosure.add_part_to_face(Face.BOTTOM, "Support SPST 3", pf.build_support(support_height=17), abs_pos=(15, ENCLOSURE_INNER_LENGTH-10-ENCLOSURE_WALLS_THICKNESS*2))
# enclosure.add_part_to_face(Face.TOP, "Encoder", pf.build_encoder(), abs_pos=(200, 100))
for i in range(0, 4):
    enclosure.add_part_to_face(Face.BACK, f"Js{i}", pf.build_jack(part_type='3.5mm XXX'), abs_pos=(8, 4.5 + i*9))
for i in range(0, 7):
    enclosure.add_part_to_face(Face.BACK, f"Jb{i}", pf.build_jack(), abs_pos=(24 + i*18, 10))
    enclosure.add_part_to_face(Face.BACK, f"Jt{i}", pf.build_jack(), abs_pos=(24 + i*18, 18+9))
#enclosure.add_part_to_face(Face.BACK, "USB", pf.build_usb_a(), rel_pos=(0, 0))
#enclosure.add_part_to_face(Face.BACK, "USB V", pf.build_usb_a(orientation_vertical=True), rel_pos=(40, 0))
#enclosure.add_part_to_face(Face.BACK, "USB C", pf.build_usb_c(), rel_pos=(0, 0))
#enclosure.add_part_to_face(Face.BACK, "USB C V", pf.build_usb_c(orientation_vertical=True), rel_pos=(40, 0))
#enclosure.add_part_to_face(Face.LEFT, "Vent L", pf.build_air_vent(), rel_pos=(0, 0))
#enclosure.add_part_to_face(Face.RIGHT, "Vent R", pf.build_air_vent(), rel_pos=(0, 0))

skirt = pf.build_support(part_type="skirt", width=(enclosure_size.outer_width - enclosure_size.wall_thickness*2), length=(enclosure_size.outer_length - enclosure_size.wall_thickness*2))
enclosure.add_part_to_face(Face.TOP, "Skirt", skirt, rel_pos=(0, 0))

enclosure.assemble(walls_explosion_factor=1.0, lid_panel_shift=0)

show(enclosure.assembly, hide_contains=["Masks", "Debug", "BOTTOM", "FRONT", "BACK", "LEFT", "RIGHT"])

TODO Need to move all workplanes so the hole's center is at 0,0
VALIDATING CLASS: Hdmi5InchJrp5015Part
[My project v1.0.0] TOP: adding part 'Screen'
VALIDATING CLASS: ButtonSpstPbs24b4Part
[My project v1.0.0] TOP: adding part 'SPST 3'
VALIDATING CLASS: PyramidSupportPart
[My project v1.0.0] BOTTOM: adding part 'Support SPST 3'
VALIDATING CLASS: Jack3_5mmXxxPart
[My project v1.0.0] BACK: adding part 'Js0'
VALIDATING CLASS: Jack3_5mmXxxPart
[My project v1.0.0] BACK: adding part 'Js1'
VALIDATING CLASS: Jack3_5mmXxxPart
[My project v1.0.0] BACK: adding part 'Js2'
VALIDATING CLASS: Jack3_5mmXxxPart
[My project v1.0.0] BACK: adding part 'Js3'
VALIDATING CLASS: Jack6_35mmPj612aPart
[My project v1.0.0] BACK: adding part 'Jb0'
VALIDATING CLASS: Jack6_35mmPj612aPart
[My project v1.0.0] BACK: adding part 'Jt0'
VALIDATING CLASS: Jack6_35mmPj612aPart
[My project v1.0.0] BACK: adding part 'Jb1'
VALIDATING CLASS: Jack6_35mmPj612aPart
[My project v1.0.0] BACK: adding part 'Jt1'
VALIDATING CLASS: Jack6

In [14]:
enclosure.export_printables()

Exporting 'box' to 'stls/my_project-box-v1.0.0.stl'
Exporting 'lid' to 'stls/my_project-lid-v1.0.0.stl'


## Panel layout building

In [13]:
from cq_enclosure_builder.layout_builder import LayoutElement, LayoutGroup

In [11]:
from cq_enclosure_builder.parts.common.screw_block import ScrewBlock

screw = ScrewBlock().m3(4, is_counter_sunk=True, with_counter_sunk_block=True)
a = (
    cq.Assembly(None, name="Screw block")
        .add(screw["block"], name="Block", color=cq.Color(1, 0, 0))
        .add(screw["mask"], name="Block Mask", color=cq.Color(0, 1, 0))
        .add(screw["counter_sunk_block"], name="CS Block", color=cq.Color(0, 1, 1))
        .add(screw["counter_sunk_mask"], name="CS Block Mask", color=cq.Color(0, 1, 0))
)
show(a)

Building CS cat m3 - ref M3-0.5
100% ⋮————————————————————————————————————————————————————————————⋮ (4/4)  0.18s


In [14]:
# Fixed-width vertical line 'tests'

enclosure_size = (300, 150, 38, 2)

vertical_fixed_xg = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VF!G", usb_a), ("SPST VF!G", spst), ("Jack VF!G", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=False, elements_centers_at_0_0=True, align_to_outside_footprint=False)
vertical_fixed_xg_aligned = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VFA!G", usb_a), ("SPST VFA!G", spst), ("Jack VFA!G", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=False, elements_centers_at_0_0=True, align_to_outside_footprint=True)

vertical_fixed_xe = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VF!E", usb_a), ("SPST VF!E", spst), ("Jack VF!E", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=True, elements_centers_at_0_0=False, align_to_outside_footprint=False)
vertical_fixed_xe_aligned = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VFA!E", usb_a), ("SPST VFA!E", spst), ("Jack VFA!E", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=True, elements_centers_at_0_0=False, align_to_outside_footprint=True)

vertical_fixed = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VF", usb_a), ("SPST VF", spst), ("Jack VF", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=True, elements_centers_at_0_0=True, align_to_outside_footprint=False)
vertical_fixed_aligned = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VFA", usb_a), ("SPST VFA", spst), ("Jack VFA", jack_635)], horizontal=False,
    add_margin_on_sides=False, group_center_at_0_0=True, elements_centers_at_0_0=True, align_to_outside_footprint=True)

vertical_fixed_margin = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VFM", usb_a), ("SPST VFM", spst), ("Jack VFM", jack_635)], horizontal=False,
    add_margin_on_sides=True, group_center_at_0_0=True, elements_centers_at_0_0=True, align_to_outside_footprint=False)
vertical_fixed_margin_aligned = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[1], [("USB VFMA", usb_a), ("SPST VFMA", spst), ("Jack VFMA", jack_635)], horizontal=False,
    add_margin_on_sides=True, group_center_at_0_0=True, elements_centers_at_0_0=True, align_to_outside_footprint=True)

enclosure = Enclosure(*enclosure_size)

for idx, elem in enumerate(vertical_fixed_xg.translate((0, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())
for idx, elem in enumerate(vertical_fixed_xg_aligned.translate((30, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())

for idx, elem in enumerate(vertical_fixed_xe.translate((-60, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())
for idx, elem in enumerate(vertical_fixed_xe_aligned.translate((-30, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())

for idx, elem in enumerate(vertical_fixed.translate((20, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))
for idx, elem in enumerate(vertical_fixed_aligned.translate((50, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))
    
for idx, elem in enumerate(vertical_fixed_margin.translate((100, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))
for idx, elem in enumerate(vertical_fixed_margin_aligned.translate((130, 0)).get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))

enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks", "FRONT", "RIGHT", "BOTTOM"])

TOP: adding part 'USB VF!G'
TOP: adding part 'SPST VF!G'
TOP: adding part 'Jack VF!G'
TOP: adding part 'USB VFA!G'
TOP: adding part 'SPST VFA!G'
TOP: adding part 'Jack VFA!G'
TOP: adding part 'USB VF!E'
TOP: adding part 'SPST VF!E'
TOP: adding part 'Jack VF!E'
TOP: adding part 'USB VFA!E'
TOP: adding part 'SPST VFA!E'
TOP: adding part 'Jack VFA!E'
TOP: adding part 'USB VF'
TOP: adding part 'SPST VF'
TOP: adding part 'Jack VF'
TOP: adding part 'USB VFA'
TOP: adding part 'SPST VFA'
TOP: adding part 'Jack VFA'
TOP: adding part 'USB VFM'
TOP: adding part 'SPST VFM'
TOP: adding part 'Jack VFM'
TOP: adding part 'USB VFMA'
TOP: adding part 'SPST VFMA'
TOP: adding part 'Jack VFMA'
100% ⋮————————————————————————————————————————————————————————————⋮ (158/158) 19.29s


In [16]:
g00 = True
e00 = True
astof = False

add_margin = False
align_all = False

top_line = LayoutGroup.line_of_parts([("USB", usb_a), ("SPST", spst), ("Jack", jack_635)],
                                   margin=0, horizontal=False,
                                   group_center_at_0_0=False, elements_centers_at_0_0=e00,
                                   align_start_to_outside_footprint=astof)

side_line = LayoutGroup.line_of_parts([("USB Hz", usb_a), ("Jack Hz", jack_635)],
                                   margin=0, horizontal=True,
                                   group_center_at_0_0=g00, elements_centers_at_0_0=e00,
                                   align_start_to_outside_footprint=astof)

side_line_fixed = LayoutGroup.fixed_width_line_of_parts(ENCLOSURE_INNER_WIDTH, [("USB Fix Hz", usb_a), ("Jack Fix Hz", jack_635)],
                                                   horizontal=True,
                                                   add_margin_on_sides=add_margin,
                                                   group_center_at_0_0=True,
                                                   elements_centers_at_0_0=e00,
                                                   align_to_outside_footprint=align_all)

enclosure = Enclosure(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_OUTER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
top_line.translate((20, 0))
for idx, elem in enumerate(top_line.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())
for idx, elem in enumerate(side_line.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())

for idx, elem in enumerate(side_line_fixed.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks"])

TOP: adding part 'USB'
TOP: adding part 'SPST'
TOP: adding part 'Jack'
TOP: adding part 'USB Hz'
TOP: adding part 'Jack Hz'
TOP: adding part 'USB Fix Hz'
TOP: adding part 'Jack Fix Hz'
100% ⋮————————————————————————————————————————————————————————————⋮ (52/52)  7.01s


In [46]:
top_line = LayoutGroup.line_of_parts([("Jack1", jack_635), ("Jack2", jack_635), ("Jack£", jack_635)],
                                   margin=10, horizontal=False,
                                   group_center_at_0_0=True, elements_centers_at_0_0=True,
                                   align_start_to_outside_footprint=False)

enclosure = Enclosure(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_OUTER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
for idx, elem in enumerate(top_line.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks"])

TOP: adding part 'Jack1'
TOP: adding part 'Jack2'
TOP: adding part 'Jack£'
100% ⋮————————————————————————————————————————————————————————————⋮ (32/32)  0.28s


In [58]:
enclosure = Enclosure(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_OUTER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)

left_controls = LayoutElement("SPST", spst)
# left_controls = LayoutGroup.line_of_parts([("SPST bottom-left", spst), ("SPST top-left", spst)], group_center_at_0_0=True, margin=30, horizontal=False, align_start_to_outside_footprint=True)
screen_element = LayoutElement("Screen", screen)
right_control = LayoutElement("Encoder", encoder)
#right_controls = LayoutGroup.line_of_parts([("SPST bottom-right", spst), ("SPST top-right", spst)], margin=30, horizontal=False)
print("---")
top_panel = LayoutGroup.fixed_width_line_of_elements(
    ENCLOSURE_INNER_WIDTH,
    [left_controls, screen_element, right_control],
    # [screen_element],
    #[left_controls, screen_element, right_controls],
    horizontal=True,
    add_margin_on_sides=True,
    group_center_at_0_0=True,
    align_to_outside_footprint=True
)
print("---")

for idx, elem in enumerate(top_panel.get_elements()):
# for idx, elem in enumerate(left_controls.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_abs_pos(enclosure.panels[Face.TOP]))
#enclosure.add_part_to_face(Face.TOP, screen_element.label, screen_element.part, rel_pos=screen_element.get_pos())
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks"])

---
---
TOP: adding part 'SPST'
TOP: adding part 'Screen'
TOP: adding part 'Encoder'
100% ⋮————————————————————————————————————————————————————————————⋮ (38/38)  1.33s


In [56]:
top_parts = [("Upper SPST", spst), usb, jack_635, usb_c, usb_c, encoder]
top_group = LayoutGroup.line_of_parts(top_parts, margin=2, horizontal=True, group_center_at_0_0=True)

bottom_parts = [jack_635, jack_635, jack_635, jack_35, jack_635, jack_635, jack_635, usb_c, usb_c, ("Bottom Encoder", encoder)]
bottom_group = LayoutGroup.line_of_parts(bottom_parts, margin=2, horizontal=True, group_center_at_0_0=True)

groups = [top_group, bottom_group]
groups_group = LayoutGroup.line_of_elements(groups, margin=2, horizontal=False)

print("gg's TOTAL footprint: " + str(groups_group.total_footprint[0]) + "," + str(groups_group.total_footprint[1]))
print("gg's outside footprint: " + str(groups_group.outside_footprint[0]) + "," + str(groups_group.outside_footprint[1]))
print("gg's total footprint offset:" + str(groups_group.total_footprint_offset[0]) + "," + str(groups_group.total_footprint_offset[1]))


enclosure = Enclosure(ENCLOSURE_INNER_WIDTH, ENCLOSURE_INNER_LENGTH, ENCLOSURE_OUTER_THICKNESS, ENCLOSURE_WALLS_THICKNESS)
for idx, elem in enumerate(groups_group.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks"])

gg's TOTAL footprint: 185.475,65.69999999999999
gg's outside footprint: 185.475,63.1
gg's total footprint offset:0,0
TOP: adding part 'Upper SPST'
TOP: adding part '34a04749-0991-4fa3-924e-6fd88412df74'
TOP: adding part '22a42a92-9ae8-45a3-b420-f8070c78e151'
TOP: adding part '9d070e1b-28d2-446f-8f17-999714581cd8'
TOP: adding part '4ba9b359-6f46-4aa3-a374-f1ab39a5dc41'
TOP: adding part '83e62d6f-8455-4f0d-ad48-13dc4331d113'
TOP: adding part 'dc450d4b-2da8-40c6-aedf-b7f7a86ec18c'
TOP: adding part 'a88064bd-161b-4dab-8902-70df1dbee816'
TOP: adding part 'e0504682-0034-485f-b47d-bc31fce83da6'
TOP: adding part 'dfac720e-7c10-4654-8916-c3f9c008ffb9'
TOP: adding part 'ba59dbc8-db14-4808-be68-7ac280369a78'
TOP: adding part '95b156ae-9512-4ec0-9630-3912bf7e684a'
TOP: adding part '923d18de-f2b6-4bee-bd7e-2714a3b682ea'
TOP: adding part '5b475095-85e0-42a8-9571-a715723e08fc'
TOP: adding part '9655e916-8e53-447d-bfa4-305676c7faa4'
TOP: adding part 'Bottom Encoder'
100% ⋮—————————————————————————————

In [53]:

enclosure_size = (300, 150, 38, 2)
enclosure = Enclosure(*enclosure_size)

jacks_grid_1 = LayoutGroup.grid_of_part("Jack 6.35", jack_635, rows=2, cols=5, margin_rows=5, margin_cols=2)
jacks_grid_2 = LayoutGroup.grid_of_part("Jack 3.5", jack_35, rows=4, cols=3, margin_rows=1, margin_cols=1)
one_usb = LayoutElement("USB", usb_a)

all_jacks = LayoutGroup.line_of_elements(
    [jacks_grid_1, jacks_grid_2, one_usb],
    margin=10,
    elements_centers_at_0_0=True)

some_fixed_width_line = LayoutGroup.fixed_width_line_of_parts(
    enclosure_size[0],
    [("USB VFA!G", usb_a), ("SPST VFA!G", spst), ("Jack VFA!G", jack_635)],
    horizontal=True,
    add_margin_on_sides=False, group_center_at_0_0=False, elements_centers_at_0_0=True, align_to_outside_footprint=True)

for idx, elem in enumerate(all_jacks.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, rel_pos=elem.get_pos())
for idx, elem in enumerate(some_fixed_width_line.get_elements()):
    enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())
enclosure.assemble()

show(enclosure.assembly, hide_contains=["Masks"])

TOP: adding part 'Jack 6.35 1-1'
TOP: adding part 'Jack 6.35 1-2'
TOP: adding part 'Jack 6.35 1-3'
TOP: adding part 'Jack 6.35 1-4'
TOP: adding part 'Jack 6.35 1-5'
TOP: adding part 'Jack 6.35 2-1'
TOP: adding part 'Jack 6.35 2-2'
TOP: adding part 'Jack 6.35 2-3'
TOP: adding part 'Jack 6.35 2-4'
TOP: adding part 'Jack 6.35 2-5'
TOP: adding part 'Jack 3.5 1-1'
TOP: adding part 'Jack 3.5 1-2'
TOP: adding part 'Jack 3.5 1-3'
TOP: adding part 'Jack 3.5 2-1'
TOP: adding part 'Jack 3.5 2-2'
TOP: adding part 'Jack 3.5 2-3'
TOP: adding part 'Jack 3.5 3-1'
TOP: adding part 'Jack 3.5 3-2'
TOP: adding part 'Jack 3.5 3-3'
TOP: adding part 'Jack 3.5 4-1'
TOP: adding part 'Jack 3.5 4-2'
TOP: adding part 'Jack 3.5 4-3'
TOP: adding part 'USB'
TOP: adding part 'USB VFA!G'
TOP: adding part 'SPST VFA!G'
TOP: adding part 'Jack VFA!G'
100% ⋮————————————————————————————————————————————————————————————⋮ (168/168)  5.71s


## Test prints

### All screwable parts

In [15]:
enclosure_wall_thickness = pf.default_parameters["enclosure_wall_thickness"]

width = 160
length = 25

parts = [
    ("Pot", pot),
    ("Banana", banana),
    ("Barrel", barrel_plug),
    ("RCA", rca),
    ("SPST", spst),
    ("Encoder", encoder),
    ("Jack 3.5", jack_35),
    ("Jack 6.35", jack_635),
]

line = LayoutGroup.fixed_width_line_of_parts(
    width, parts,
    horizontal=True,
    add_margin_on_sides=True,
    group_center_at_0_0=True,
    elements_centers_at_0_0=True,
    align_to_outside_footprint=True)


test_panel = Panel(Face.TOP, PanelSize(width, length, enclosure_wall_thickness))

for idx, elem in enumerate(line.translate((0, 0)).get_elements()):
    test_panel.add(elem.label, elem.part, rel_pos=elem.get_pos())
    # enclosure.add_part_to_face(Face.TOP, elem.label, elem.part, abs_pos=elem.get_pos())

test_panel.assemble()


# show(test_panel.debug_assemblies["combined"])
show(test_panel.panel)

from cadquery import exporters
#exporters.export(test_panel.panel.toCompound(), 'models/all-threads-v3.stl')

NameError: name 'LayoutGroup' is not defined